In [1]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
def get_continuous_chunks(tagged_sent):
    continuous_chunk = []
    current_chunk = []

    for token, tag in tagged_sent:
        if tag != "O":
            current_chunk.append((token, tag))
        else:
            if current_chunk: # if the current chunk is not empty
                continuous_chunk.append(current_chunk)
                current_chunk = []
    # Flush the final current_chunk into the continuous_chunk, if any.
    if current_chunk:
        continuous_chunk.append(current_chunk)
    return continuous_chunk

In [3]:
from stanza.text.vocab import Vocab

In [4]:
from stanza.nlp.corenlp import CoreNLPClient

In [5]:
client = CoreNLPClient(server='http://localhost:9000', default_annotators=['ssplit', 'tokenize', 'lemma', 'pos', 'ner'])

In [87]:
# annotated = client.annotate('Liverpol of Receipt 171955 Local June 4 1994 Ang Unusual Meteorological Conditions Nil')

annotated = client.annotate('a black and white photo of a plane flying in the sky .')

In [88]:
annotated.sentences

[[Sentence: a black and white photo of a plane flying in the s]]

In [89]:
for sentence in annotated.sentences:
#     print('sentence', sentence)
    ne_tagged_sent = [(token.word, token.ner) for token in sentence]
    named_entities = get_continuous_chunks(ne_tagged_sent)
    named_entities = get_continuous_chunks(ne_tagged_sent)
    named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]
    print named_entities_str_tag

[]


In [9]:
[(token.word, token.ner) for token in sentence]

[(u'Liverpol', u'O'),
 (u'of', u'O'),
 (u'Receipt', u'O'),
 (u'171955', u'NUMBER'),
 (u'Local', u'O'),
 (u'June', u'DATE'),
 (u'4', u'DATE'),
 (u'1994', u'DATE'),
 (u'Ang', u'O'),
 (u'Unusual', u'O'),
 (u'Meteorological', u'O'),
 (u'Conditions', u'O'),
 (u'Nil', u'O')]

-----

In [11]:
import pandas as pd
import pickle
import math
import time
from dateutil.parser import parse
from geopy.geocoders import Nominatim
from datetime import datetime
geolocator = Nominatim()

In [12]:
def is_date(string):
    try: 
        parse(string)
        if string.isdigit():
            return False
        return True
    except ValueError:
        return False

In [11]:
is_date("1990-12-1")

True

In [12]:
is_date("Jan"), is_date('15')

(True, False)

In [13]:
location = geolocator.geocode("RAF United Kingdom")

In [14]:
location.latitude, location.longitude

(55.9515812, -3.3469413)

In [15]:
location.address

u'RAF, Turnhouse Road, Dolphington Cottages, Ingliston, City of Edinburgh, Scotland, EH12 0AW, UK'

In [199]:
df = pd.read_csv('ufo_hw2_update.csv')

In [200]:
len(df)

3533

In [17]:
df.head(4)

,Captions,Class,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Id,URL,id,text
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFE-24-1922_29,"K. NEARBY OBJECTS\n(Telephone lines, high volt..."
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFE-24-1922_15,REPORT OF AN UNIDENTIFIED FLYING OBJECT\n\n \n...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFE-24-1922_114,K. m:nm OBJECTS - lewga 7 ( ' ‘\n(Telephone li...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFE-24-1922_100,Reply To:\n\nr.\n\n \n \n \n \n \n\n\( .\n\nYo...


In [18]:
df['location'] = ''
df['date'] = ''
df['lat'] = ''
df['lon'] = ''
df['named_entities'] = ''

In [ ]:
master_dict = {}
errors_ids = []
for index, row in df.iterrows():
    try:
        # skip image entries for now...
        if 'DEFE' not in str(row['id']):
            continue

        text = row['text']
        annotated = client.annotate(str(text))
        print index
        ners = []
        for sentence in annotated.sentences:
    #     print('sentence', sentence)
            ne_tagged_sent = [(token.word, token.ner) for token in sentence]
            named_entities = get_continuous_chunks(ne_tagged_sent)
            named_entities = get_continuous_chunks(ne_tagged_sent)
            named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]
            ners.extend(named_entities_str_tag)
        ners = [tuple(reversed(x)) for x in ners]
        master_dict[row['id']] = ners
        row_location = ''
        row_lat = ''
        row_lon = ''
        for nr in ners:
            if nr[0] == 'LOCATION':
                location = geolocator.geocode(nr[1] + " United Kingdom")
                if location:
                    if 'UK' in location.address:
                        row_location = location
                        row_lat = location.latitude
                        row_lon = location.longitude
                        df.loc[index, 'location'] = location.address
                        df.loc[index, 'lat'] = row_lat
                        df.loc[index, 'lon'] = row_lon
                        break

        time.sleep(6.5)
        # if we cant get location NE, try to get from organization
        if row_location == '':
            for nr in ners:
                if nr[0] == 'ORGANIZATION':
                    location = geolocator.geocode(nr[1] + " United Kingdom")
                    if location:
                        if 'UK' in location.address:
                            row_location = location
                            row_lat = location.latitude
                            row_lon = location.longitude
                            df.loc[index, 'location'] = location.address
                            df.loc[index, 'lat'] = row_lat
                            df.loc[index, 'lon'] = row_lon
                            break
        row_date = ''
        for nr in ners:
            if nr[0] == 'DATE':
                if is_date(nr[1]):
                    if is_date(row_date+nr[1]):
                        row_date += nr[1]


        df.loc[index, 'date'] = row_date
        df.loc[index, 'named_entities'] = str(ners)
    #     if index == 1:
    #         break
    except:
        errors_ids.append(index)
filename = str(datetime.now()) + ".pickle"
filename_df = str(datetime.now()) + "_df.pickle"
with open(filename.replace(" ", "-").replace("/","-"), 'wb') as handle:
    pickle.dump(master_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(filename_df.replace(" ", "-").replace("/","-"), 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('filename.pickle', 'rb') as handle:
#     master_dict = pickle.load(handle)

In [21]:
len(master_dict)

1976

In [27]:
header = ['id','location','lat','lon','DATE', 'text', 'named_entities']

In [28]:
df.to_csv('pdfs_with_ner.csv', index=False, columns=header)

-----

In [99]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.chunk import conlltags2tree
from nltk.tree import Tree

In [33]:
st = StanfordNERTagger('/Users/sanjay/Downloads/stanford-ner-2018-02-27/classifiers/english.conll.4class.distsim.crf.ser.gz', '/Users/sanjay/Downloads/stanford-ner-2018-02-27/stanford-ner.jar')

In [63]:
# text = open('/Users/sanjay/Desktop/Masters/SPRING18/CS599/UFO_ocr_image-captioning_enrichment/british-ufo-files/DEFE-31-175/outtxt/10.txt','r') .read()
text = 'It took 50 seconds'

In [39]:
tokenized_text = word_tokenize(text)
ne_tagged_sent = st.tag(tokenized_text)

In [40]:
tokenized_text

['It', 'took', '50', 'seconds']

In [41]:
ne_tagged_sent

[(u'It', u'O'), (u'took', u'O'), (u'50', u'O'), (u'seconds', u'O')]

In [64]:
annotated = client.annotate(text)

In [65]:
annotated.sentences

[[Sentence: It took 50 seconds]]

In [66]:
for sentence in annotated.sentences:
    print('sentence', sentence)
    for token in sentence:
        print(token.word, token.lemma, token.pos, token.ner)

In [68]:
sentence = annotated.sentences[0]

In [69]:
sentence

[Sentence: It took 50 seconds]

In [71]:
for token in sentence:
    print(token.word, token.lemma, token.pos, token.ner)

In [74]:
sentence

[Sentence: It took 50 seconds]

In [78]:
sentence[1].ner

u'O'

----

#### Add duration to csv

In [160]:
df = pd.read_csv('pdfs_with_ner.csv')
df['duration'] = ''

In [161]:
# make sure you're opening up the pickled master_dict from the previous section
with open('2018-04-1211-04-33.411746.pickle', 'rb') as handle:
    master_dict = pickle.load(handle)

In [162]:
for index, row in df.iterrows():
    if 'DEFE' not in str(row['id']):
        continue

    ners = master_dict[row['id']]
    ners_dict = dict(ners)
    if 'DURATION' in ners_dict:
#         print ners_dict['DURATION']
        df.loc[index, 'duration'] = ners_dict['DURATION']
        continue
df.columns = df.columns.str.strip()

In [163]:
df.to_csv('pdfs_with_ner_duration.csv',index=False)

In [164]:
df.head(2)

,id,location,lat,lon,date,text,named_entities,duration
0,DEFE-24-1922_29,"RAF, Turnhouse Road, Dolphington Cottages, Ing...",55.951581,-3.346941,MAY,"K. NEARBY OBJECTS\r(Telephone lines, high volt...","[(u'TITLE', u'river'), (u'CAUSE_OF_DEATH', u'f...",
1,DEFE-24-1922_15,"Time, Telegraph Road, Woodchurch, Wirral, Nort...",53.328708,-3.100154,NaN,REPORT OF AN UNIDENTIFIED FLYING OBJECT\r\r \r...,"[(u'ORGANIZATION', u'Time'), (u'NUMBER', u'27,...",


----

#### Merge captioned images dataset with corresponding lat/lon/summary

In [179]:
# images_df = pd.read_csv('ufo_stalker_urls_v5.csv')
images_df = pd.read_csv('captions_classes_with_summary.csv')

images_df = images_df[['URL', 'lat', 'lon', 'Country', 'Summary']]

In [180]:
images_df.head(3)

,URL,lat,lon,Country,Summary
0,http://www.mufoncms.com/files_jeud8334j/91189_...,39.699956,-105.4162,United States,Caught in a photo of the full moon. i could n...
1,http://www.mufoncms.com/files_jeud8334j/91189_...,39.699956,-105.4162,United States,Caught in a photo of the full moon. i could n...
2,http://www.mufoncms.com/files_jeud8334j/91189_...,39.699956,-105.4162,United States,Caught in a photo of the full moon. i could n...


In [181]:
images_df = images_df.rename(index=str, columns={'Summary': 'description'})

In [182]:
# the captioned images file
captioned_images_df = pd.read_csv('data_set_v2_caption.csv')

In [183]:
captioned_images_df.head()

,Captions,Case_ID,Class,URL
0,a black and white photo of a plane flying in t...,4023,"['parachute, chute']",http://localhost:8764/inception/v3/caption/ima...
1,a view of a city street from the side of the r...,4783,['balloon'],http://localhost:8764/inception/v3/caption/ima...
2,a red fire hydrant sitting in the middle of a ...,4783,['window screen'],http://localhost:8764/inception/v3/caption/ima...
3,a plane flying over a large body of water .,4783,"['beacon, lighthouse, beacon light, pharos']",http://localhost:8764/inception/v3/caption/ima...
4,a collage of photos with a person holding a fr...,4028,"['web site, website, internet site, site']",http://localhost:8764/inception/v3/caption/ima...


In [184]:
# get rid of http://localhost: ....
for index,row in captioned_images_df.iterrows():
    url = row['URL'].split("http://localhost:8764/inception/v3/caption/image?url=",1)[1]
    captioned_images_df.loc[index, 'URL'] = url

In [188]:
len(captioned_images_df.merge(images_df, on='URL', how='left'))

751

In [189]:
len(captioned_images_df)

751

----

#### This shows that NER on image captions is useless

In [95]:
for index,row in captioned_images_df.iterrows():
    text = row['Captions']
    annotated = client.annotate(str(text))
    ners = []
    for sentence in annotated.sentences:
    #     print('sentence', sentence)
        ne_tagged_sent = [(token.word, token.ner) for token in sentence]
        named_entities = get_continuous_chunks(ne_tagged_sent)
        named_entities = get_continuous_chunks(ne_tagged_sent)
        named_entities_str_tag = [(" ".join([token for token, tag in ne]), ne[0][1]) for ne in named_entities]
        ners.extend(named_entities_str_tag)
    ners = [tuple(reversed(x)) for x in ners]
    if ners:
        print index,ners

2 [(u'CAUSE_OF_DEATH', u'fire')]
41 [(u'CRIMINAL_CHARGE', u'train'), (u'CRIMINAL_CHARGE', u'train')]
49 [(u'TIME', u'night'), (u'TIME', u'night')]


----

## concat with images dataset

In [192]:
captioned_images_df = pd.read_csv('data_set_v2_caption.csv')
captioned_images_df['id'] = ''

In [193]:
captioned_images_df.head(2)

,Captions,Case_ID,Class,URL,id
0,a black and white photo of a plane flying in t...,4023,"['parachute, chute']",http://localhost:8764/inception/v3/caption/ima...,
1,a view of a city street from the side of the r...,4783,['balloon'],http://localhost:8764/inception/v3/caption/ima...,


In [194]:
for index,row in captioned_images_df.iterrows():
    row_id = row['URL'].split("http://localhost:8764/inception/v3/caption/image?url=http://www.mufoncms.com/",1)[1].replace('/','_')
    captioned_images_df.loc[index, 'id'] = row_id
        

In [195]:
len(captioned_images_df), len(df)

(751, 1976)

In [196]:
list(df)

['id', 'location', 'lat', 'lon', 'date', 'text', 'named_entities', 'duration']

In [197]:
len(captioned_images_df.merge(df[['id', 'location', 'lat', 'lon', 'date', 'named_entities', 'duration']], on='id', how='outer'))

2727

In [198]:
captioned_images_df.merge(df, on='id', how='outer').to_csv('final_not_really.csv',index=False, encoding='ISO-8859-1')

UnicodeDecodeError: 'utf8' codec can't decode byte 0xe4 in position 11860: invalid continuation byte

In [178]:
len(pd.read_csv('final_not_really.csv'))

4233

In [122]:
captioned_images_df.loc[0,'Captions']

'a black and white photo of a plane flying in the sky .'

In [157]:
df.columns = df.columns.str.strip()

## Add shape from all UFO records

In [116]:
common_shapes = ['changed','light','shape','fireball','egg','round','cylinder','unknown','pyramid','sphere','cone','oval','disk','flare','unknown','crescent','flash','cross','other','teardrop','chevron','circle','rectangle','triangle','cigar','diamond','delta','formation','changing']